In [5]:
pip install -U cython

  Using cached https://files.pythonhosted.org/packages/2b/4d/899fb0967be8b02bdbc47f892b48f576f9c2d28f7d49710bcecf9ed1c705/Cython-0.29.21-cp37-cp37m-macosx_10_9_x86_64.whl
  Found existing installation: Cython 0.29.13
    Uninstalling Cython-0.29.13:
      Successfully uninstalled Cython-0.29.13
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install guidedlda

  Using cached https://files.pythonhosted.org/packages/f8/ee/6d6e2b3525388399e12a4482554c7529a5fcf5e99c50a60abaa02894b8bf/guidedlda-2.0.0.dev22.tar.gz
  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-install-e5rrdo79/guidedlda/setup.py'"'"'; __file__='"'"'/private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-install-e5rrdo79/guidedlda/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-wheel-lmwis45r --python-tag cp37
       cwd: /private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-install-e5rrdo79/guidedlda/
  Complete output (152 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creat

Failed to build guidedlda
  Running setup.py install for guidedlda ... error
    ERROR: Command errored out with exit status 1:
     command: /opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-install-e5rrdo79/guidedlda/setup.py'"'"'; __file__='"'"'/private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-install-e5rrdo79/guidedlda/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-record-qsrzmq8o/install-record.txt --single-version-externally-managed --compile
         cwd: /private/var/folders/tw/8ygs2pd16g7glmkjg1xgxk9c0000gn/T/pip-install-e5rrdo79/guidedlda/
    Complete output (154 lines):
    running install
    [pbr] Generating AUTHORS
    [pbr] AUTHORS complete (0.0s)
    running bu

Note: you may need to restart the kernel to use updated packages.


In [10]:
#read data(exclude replies) 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
import pandas as pd

text = pd.read_csv('Jan6(excludes replies).csv')
print('The number of tweets (excludes replies):',len(text))

The number of tweets (excludes replies): 6501


In [11]:
#Text cleaning
import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words += ['cultured','clean','cultivated']
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space.
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if token not in stop_words:
            token = porter.stem(token) #root form
            doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]

In [33]:
table2 = pd.read_excel('Table 1,2,3 Jan 6.xlsx',sheet_name=1)
table2

,Theme,Bigrams within topics
0,Environmental Impact,"reduce environment, damage environment, enviro..."
1,NaN,"without environment, environmental issues, car..."
2,NaN,"greenhouse emission,"
3,NaN,envoronmental ipmact
4,NaN,champion environment
...,...,...
104,Market,"meat market, consumer worldwild"
105,NaN,"hit market, product market"
106,NaN,road market
107,NaN,"bring market, give consumer"


In [42]:
#unigram
unigram = []
unigram_list = []
for index, i in text.iterrows():
    unigram=''
    for word in i['processed'].split():
        unigram+= word+' '
    unigram_list.append(unigram)


import guidedlda
import numpy as np
import gensim

seed_topic_list = [['environ','environment','carbon','climat','greenhous','emiss','animal','without','slaughter',
                   'cruelti','welfar','health','healthier','benefit','public','protein'],# Benefit
                  ['antibiot','antimicrobi','resist','safeti','contamin','consumpt','popul',
                  'demand','secur','wast','industri','suppli','convent','regular','system'], #Food System
                  ['seafood','sea','ocean','fish','aquacultur','chicken','duck','poultri','egg'], #Product(seafood and poultry)
                  ['delici','tast','tender','textur','juici','test'],#Taste and Texture
                  ['cell','stem','cultur','muscl','biolog','divis'], # Process
                  ['seri','fund','funder','rais','invest','investor','dollar','pound','announc', 
                  'happi','thrill','excit'], #Announcement
                  ['thank','thankyou','support','shoutout','help'], #Appreciation
                  ['confer','regist','live','symposium','stream','livestream'], #conference and summit
                  ['market','consum','store','groceri'], # Market
                  ['plant','vegan','vegetarian'], #Vegetarian
                  ['check','post','paper','interview','articl','blog','news','break'], #Media
                  ['hire','join','team']]#Hiring Information


In [ ]:
'''Version 1: According to Table 2, we divide bi-grams into unigrams, and remove some unigrams that might occur in
different themes. For example, we devide 'meat_comsumption' into 'meat' and 'consumption', but 'meat' will not only
occur in Theme'Food Security', but also occur in some other themes. Therefore, we won't consider 'meat' as a seed word '''

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

model = guidedlda.GuidedLDA(n_topics=13,n_iter=200,random_state=1,refresh=10,alpha=0.1,eta=0.01)
vectorizer = CountVectorizer(min_df = 2)
X = vectorizer.fit_transform(text['processed'])

vocab = vectorizer.get_feature_names()
word2id = dict((v,idx) for idx,v in enumerate(vocab))
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

model.fit(X.toarray(),seed_topics=seed_topics,seed_confidence=0.25)
topic_word = model.topic_word_
n_top_words = 30
vocab = tuple(vocab)

for i, topic_dist in enumerate(topic_word):
    print('\n')
    print('Topic:',i)
    words_probability = np.array(-topic_dist)
    for index in range(n_top_words):
        print(round(abs(np.sort(words_probability))[:(n_top_words)][index],4),'*',
              np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1][index],sep='',end=' ')

X = X.toarray()
doc_topic = model.transform(X)
topic_number = []
number = []
for i in range(len(doc_topic)):
    topic_number.append(doc_topic[i].argmax())
    number.append('1')
data = pd.DataFrame(data=[i for i in topic_number],columns=['topic_number'])
data['number'] = [i for i in number]
print(data.groupby('topic_number')['number'].count())

INFO:guidedlda:n_documents: 6501
INFO:guidedlda:vocab_size: 3621
INFO:guidedlda:n_words: 62110
INFO:guidedlda:n_topics: 13
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -720288
INFO:guidedlda:<10> log likelihood: -504564
INFO:guidedlda:<20> log likelihood: -489253
INFO:guidedlda:<30> log likelihood: -481607
INFO:guidedlda:<40> log likelihood: -477747
INFO:guidedlda:<50> log likelihood: -475822
INFO:guidedlda:<60> log likelihood: -473691
INFO:guidedlda:<70> log likelihood: -472094
INFO:guidedlda:<80> log likelihood: -470580
INFO:guidedlda:<90> log likelihood: -469512
INFO:guidedlda:<100> log likelihood: -469205
INFO:guidedlda:<110> log likelihood: -468471
INFO:guidedlda:<120> log likelihood: -467762
INFO:guidedlda:<130> log likelihood: -467713
INFO:guidedlda:<140> log likelihood: -467348
INFO:guidedlda:<150> log likelihood: -467245
INFO:guidedlda:<160> log likelihood: -466520
INFO:guidedlda:<170> log likelihood: -466933
INFO:guidedlda:<180> log likelihood: -466658
INFO:g



Topic: 0
0.0451*meat 0.0354*anim 0.0193*food 0.0159*product 0.0155*health 0.0139*global 0.0132*billion 0.0118*sustain 0.0116*protein 0.0114*agricultur 0.0112*use 0.011*altern 0.0108*could 0.0106*environment 0.0103*peopl 0.0099*year 0.0099*plant 0.0087*human 0.0087*industri 0.0087*vegan 0.0085*produc 0.0085*water 0.0083*impact 0.0077*demand 0.0077*environ 0.0074*land 0.0072*feed 0.007*less 0.0068*solut 0.0068*milk 

Topic: 1
0.0191*farm 0.0177*product 0.0177*scale 0.0167*meat 0.0114*antibiot 0.0107*larg 0.01*facil 0.01*public 0.009*shrimp 0.0084*today 0.008*call 0.008*ye 0.008*factori 0.0074*move 0.0074*safe 0.007*process 0.0067*temiz 0.0067*like 0.0067*say 0.0064*human 0.0064*program 0.0064*usda 0.006*anyon 0.006*market 0.006*acceler 0.0057*keep 0.0057*found 0.0057*manufactur 0.0057*ba 0.0054*whole 

Topic: 2
0.0372*fish 0.0281*seafood 0.0248*sustain 0.0207*ocean 0.0184*product 0.015*make 0.0129*cell 0.0124*grow 0.0112*salmon 0.0098*delici 0.0095*healthi 0.0088*wild 0.0086*work 0.007

topic_number
0     694
1     245
2     339
3     522
4     437
5     465
6     631
7     686
8     488
9     677
10    448
11    394
12    475
Name: number, dtype: int64
